# Installation

In [2]:
!pip3 install tensorflow_text>=2.0.0rc0
!pip install torch_geometric
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install sentence-transformers
!pip3 install transformers

Looking in indexes: https://download.pytorch.org/whl/cu118


# Import packages

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from sentence_transformers import SentenceTransformer
from transformers import BertModel, BertTokenizer

# Import Dataset

In [4]:
cyberbully_df = pd.read_csv("cyberbully.csv")
cyberbully_df.head(3)

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN
1,1.jpg,Aaloo ke paranthe is the best breakfast Omelet...,Nonbully,Nonbully,Nonbully,Neutral,Other,No,Harmless,None,NaN,NaN
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society,NaN,NaN


# Embedding the sentence with sBERT

In [ ]:
from transformers import AutoModel, AutoTokenizer
model_name = "paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

In [7]:
embedded_sentence = []
for sentence in cyberbully_df["Img_Text"]:
    try:
        embeddings = model.encode(sentence, convert_to_tensor=True)
        embedded_sentence.append(embeddings)
    except TypeError:
        continue

# Additional node features

In [ ]:
additional_node_features = []